In [1]:
import pandas as pd
import pymongo
from sqlalchemy import create_engine
import numpy as np

In [2]:
# Initialize PyMongo to work with MongoDBs
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

In [3]:
def randomIndex(low=0,high=10,size=10):
    index_list =[]
    while len(index_list) != size:
        num=np.random.randint(low, high,1)
        if num not in index_list:
            index_list.append(num[0])
    return sorted(list((index_list)))
        
def col_to_list(col):
    return list(col.find()), len(list(col.find()))

In [4]:
quotes_mdb=client.quotes_db
author_info_col=quotes_mdb.author_information_collection


author_list,len_author_list=col_to_list(author_info_col)

author_info_df=pd.DataFrame(author_list,index=randomIndex(100,300,len_author_list))


author_info_table = author_info_df[['name','born','description']].copy()
author_info_table.index.name='author_id'
author_info_table=author_info_table.drop_duplicates(subset=['name'])
author_info_table #sql table 1

,name,born,description
author_id,,,
102,Albert Einstein,"March 14, 1879 in Ulm, Germany","\n In 1879, Albert Einstein was born in..."
103,J.K. Rowling,"July 31, 1965 in Yate, South Gloucestershire, ...",\n See also: Robert GalbraithAlthough s...
108,Jane Austen,"December 16, 1775 in Steventon Rectory, Hampsh...",\n Jane Austen was an English novelist ...
109,Marilyn Monroe,"June 01, 1926 in The United States",\n Marilyn Monroe (born Norma Jeane Mor...
114,André Gide,"November 22, 1869 in Paris, France",\n André Paul Guillaume Gide was a Fren...
116,Thomas A. Edison,"February 11, 1847 in Milan, Ohio, The United S...",\n Thomas Alva Edison was an American i...
117,Eleanor Roosevelt,"October 11, 1884 in The United States",\n Anna Eleanor Roosevelt was an Americ...
118,Steve Martin,"August 14, 1945 in Waco, Texas, The United States","\n Stephen Glenn ""Steve"" Martin is an A..."
122,Bob Marley,"February 06, 1945 in Nine Mile, Saint Ann, Jam...","\n Robert ""Bob"" Nesta Marley OM was a J..."


In [5]:
tags_col=quotes_mdb.tags_collection
tags_list,len_tags_list=col_to_list(tags_col)
tags_df=pd.DataFrame(tags_list,index=randomIndex(301,500,len_tags_list))

tags_table = tags_df[['tag']].copy()
tags_table=tags_table.rename(columns={'tag':'tag_name'})
tags_table.index.name = 'tag_id'
tags_table #sql table 2

,tag_name
tag_id,
301,change
302,deep-thoughts
303,thinking
304,world
305,abilities
...,...
492,difficult
493,grown-ups
494,write


#### 

In [6]:
quotes_col=quotes_mdb.tag_relation_collection
quotes_list,len_quotes_list=col_to_list(quotes_col)
quotes_df= pd.DataFrame(quotes_list,index=randomIndex(501,700,len_quotes_list))
# quotes_df

In [7]:
quotes_and_tag_table = quotes_df[["quote_text",'tag']].copy()
quotes_and_tag_table=quotes_and_tag_table.rename(columns={'tag':'tag_list'})
quotes_and_tag_table.index.name = 'quote_id'
# quotes_and_tag_table

In [8]:
quoteid_tag=[]

for index, row in quotes_and_tag_table.iterrows():
    for tag in row['tag_list']:
        quoteid_tag.append((index,tag))
quote_tag_df=pd.DataFrame(quoteid_tag,columns=['quote_id','tag_name'])


In [9]:
# quote_tag_df

In [10]:
quotes_and_tag_relations_table=tags_table.reset_index().merge(quote_tag_df,on='tag_name')
quotes_and_tag_relations_table=quotes_and_tag_relations_table[['tag_id','quote_id']].copy()

In [11]:
quotes_and_tag_relations_table #sql table 3

,tag_id,quote_id
0,301,503
1,302,503
2,303,503
3,303,641
4,304,503
...,...,...
227,492,692
228,493,692
229,494,692
230,497,692


In [12]:
quotes_table=quotes_and_tag_table[['quote_text']].copy()
quotes_table

,quote_text
quote_id,
503,“The world as we have created it is a process ...
506,"“It is our choices, Harry, that show what we t..."
508,“There are only two ways to live your life. On...
509,"“The person, be it gentleman or lady, who has ..."
510,"“Imperfection is beauty, madness is genius and..."
...,...
689,“You never really understand a person until yo...
692,“You have to write the book that wants to be w...
693,“Never tell the truth to people who are not wo...


In [13]:
quotes_col=quotes_mdb.quotes_collection
quotes_and_author_df=pd.DataFrame(list(quotes_col.find()))
quotes_and_author_table=quotes_and_author_df[['quote_text','author_name']].copy().rename(columns={'author_name':'name'})

In [14]:
quotes_and_author_table=quotes_and_author_table.merge(author_info_table.reset_index(), on='name')
# quotes_and_author_table

In [16]:
quotes_and_author_table=(quotes_and_author_table[['quote_text','author_id']].copy()).merge(quotes_and_tag_table.reset_index(),on="quote_text").drop(['tag_list'],axis=1)

In [21]:
quotes_and_author_table=quotes_and_author_table.set_index('quote_id')

In [22]:
quotes_and_author_table

,quote_text,author_id
quote_id,,
503,“The world as we have created it is a process ...,102
508,“There are only two ways to live your life. On...,102
511,“Try not to become a man of success. Rather be...,102
522,"“If you can't explain it to a six year old, yo...",102
544,"“If you want your children to be intelligent, ...",102
...,...,...
682,“To die will be an awfully big adventure.”,280
683,“It takes courage to grow up and become who yo...,283
688,“But better to get hurt by the truth than comf...,285
